# A demo notebook for prompt engineering best practice
Reference
- [LLM Prompting tut](https://huggingface.co/docs/transformers/v4.41.2/en/tasks/prompting)
- [Text generation strategy](https://huggingface.co/docs/transformers/v4.41.2/en/generation_strategies)

In [14]:
!pip install -q accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Decoder-only LLM
For example,  LLaMA, Llama2, Falcon, GPT2. It is often used in all other generative tasks.

In [2]:
import torch
from transformers import pipeline

/opt/conda/envs/qllava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/qllava/lib/python3.10/site-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


[2024-10-21 11:23:05,817] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
torch.manual_seed(0)
text_generator = pipeline("text-generation", model="openai-community/gpt2")

In [5]:
prompt = "Hello, I am a language model"

In [7]:
print(text_generator(prompt, max_length=50)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, I am a language modeler for Microsoft Word. I create language models using Visual C++, Microsoft SQL Server, HTML and Excel, and write a little language class with the result. It's really simple to use, with simple interface.


## Encoder-decoder LLM

In [12]:
text2text_generator = pipeline("text2text-generation", model="google/flan-t5-base")
prompt = "Translate from English to French: Hello, I am very happy to know you"

In [13]:
print(text2text_generator(prompt, max_length=50)[0]["generated_text"])

Bonjour, je suis très heureuse de vous connaître.


## Base and instruct models
Base models are excellent at completing the text when given an initial prompt.
However, they are not ideal for NLP tasks where they need to follow instructions, or for conversational use. This is where the instruct (chat) versions come in. These checkpoints are the result of further fine-tuning of the pre-trained base versions on instructions and conversational data. This additional fine-tuning makes them a better choice for many NLP tasks.

In [19]:
from transformers import pipeline, AutoTokenizer, BitsAndBytesConfig

In [17]:
model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

In [18]:
text_gen_pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    torch_dtype=torch.float16, 
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.73s/it]


## Sentiment Analysis

In [20]:
prompt = """Classify the text into neutral, negative or positive.
Text: This movie is definitely one of my favorite movies of its kind. The interaction between respectable and morally strong characters is an ode to chivalry and the honor code amongst thieves and policemen.
Sentiment:
"""

In [21]:
res = text_gen_pipeline(prompt, max_new_tokens=10)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [23]:
print(res[0]["generated_text"])

Classify the text into neutral, negative or positive.
Text: This movie is definitely one of my favorite movies of its kind. The interaction between respectable and morally strong characters is an ode to chivalry and the honor code amongst thieves and policemen.
Sentiment:
Positive


## Name entity recognition
Named Entity Recognition (NER) is a task of finding named entities in a piece of text, such as a person, location, or organization. Let’s modify the instructions in the prompt to make the LLM perform this task. Here, let’s also set return_full_text = False so that output doesn’t contain the prompt:

In [24]:
prompt = """Return a list of named entities in the text.
Text: The Golden State Warriors are a professional basketball team based in San Francisco. The Warriors compete in the National Basketball Association (NBA) as a member of the league's Western Conference Pacific Division.
Named Entities:
"""

In [31]:
sequences = text_gen_pipeline(
    prompt, 
    max_new_tokens=15, 
    return_full_text=False,  # Don't return the original prompt
)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [32]:
print(sequences[0]["generated_text"])

- Golden State Warriors
- San Francisco
- NBA
- Western Conference


## Translation

In [39]:
prompt = """Translate the English text to Italian.
Text: Sometimes, I've believe as many as six impossible things before breakfast.
Translation:
"""

In [40]:
out_seq = text_gen_pipeline(
    prompt,
    max_new_tokens=20,
    do_sample=True,
    top_k=50,
    return_full_text=False,  # Don't return the original prompt.
)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [41]:
print(out_seq[0]["generated_text"])

Piú spesso ho creduto a molte cose impossibili prima di colazione.


## Text summarization

In [46]:
prompt = """Permaculture is a design process mimicking the diversity, functionality and resilience of natural ecosystems. The principles and practices are drawn from traditional ecological knowledge of indigenous cultures combined with modern scientific understanding and technological innovations. Permaculture design provides a framework helping individuals and communities develop innovative, creative and effective strategies for meeting basic needs while preparing for and mitigating the projected impacts of climate change.
Write a summary of the above text.
Summary:
"""

In [48]:
res = text_gen_pipeline(prompt, max_new_tokens=30, do_sample=True, top_k=10, return_full_text=False)
print(res[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Permaculture is a design process mimicking natural ecosystems to meet basic needs and prepare for climate change, drawing on ecological knowledge and modern science for innovative strategies


# Question-Answering

In [49]:
prompt = """Answer the question using the context below.
Context: Gazpacho is a cold soup and drink made of raw, blended vegetables. Most gazpacho includes stale bread, tomato, cucumbers, onion, bell peppers, garlic, olive oil, wine vinegar, water, and salt. Northern recipes often include cumin and/or pimentón (smoked sweet paprika). Traditionally, gazpacho was made by pounding the vegetables in a mortar with a pestle; this more laborious method is still sometimes used as it helps keep the gazpacho cool and avoids the foam and silky consistency of smoothie versions made in blenders or food processors.
Question: What modern tool is used to make gazpacho?
Answer:
"""

In [50]:
res = text_gen_pipeline(prompt, max_new_tokens=30, do_sample=True, top_k=10, return_full_text=False)
print(res[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Blender. A blender is a modern tool often used to make gazpacho as it helps create a smooth consistency without the need of pounding the vegetables


## Reasoning

Reasoning is one of the most difficult tasks for LLMs, and achieving good results often requires applying advanced prompting techniques, like Chain-of-though.

In [69]:
#prompt = """There are 5 groups of students in the class. Each group has 4 students. How many students are there in the class?"""
prompt = """I baked 15 muffins. I ate 2 muffins and gave 5 muffins to a neighbor. My partner then bought 6 more muffins and ate 2. How many muffins do we now have?"""

In [70]:
res = text_gen_pipeline(prompt, max_new_tokens=100, do_sample=True, top_k=10, return_full_text=False)
print(res[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



We started with 15 muffins and ate 2, which means that 13 muffins are now left. Your neighbor received 5 muffins, which means we have 8 muffins remaining. After your partner ate 2, we are left with 6. So, we have 6 muffins left.


## Few-shot prompting to change the style of the generated text

In [82]:
prompt = """The first human went into space and orbited the Earth on April 12, 1961.
Date: 04/12/1961
Text: The first-ever televised presidential debate in the United States took place on September 28, 1960, between presidential candidates John F. Kennedy and Richard Nixon. 
Date:"""

In [85]:
res = text_gen_pipeline(prompt, max_new_tokens=20, do_sample=True, top_k=10, return_full_text=True)
print(res[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


The first human went into space and orbited the Earth on April 12, 1961.
Date: 04/12/1961
Text: The first-ever televised presidential debate in the United States took place on September 28, 1960, between presidential candidates John F. Kennedy and Richard Nixon. 
Date: 09/28/1960
Text:

